In [470]:
import pandas as pd

In [471]:
pd.set_option('display.max_rows', 15)

In [472]:
df_all = pd.read_csv('../data/df_all.csv').drop(['Unnamed: 0', 'id', 'cluster', 'sentiment'], axis=1)

In [473]:
df_all

,question,answer,count
0,Что вы сможете использовать в своей работе?,"формулировать проблему, 5 поче",1.0
1,Что вы сможете использовать в своей работе?,поиск потерь,1.0
2,Что вы сможете использовать в своей работе?,правило 5 «почему» :),1.0
3,Что вы сможете использовать в своей работе?,умение работать в команде,1.0
4,Что вы сможете использовать в своей работе?,картирование и анализ,1.0
...,...,...,...
29659,•Где вы хотите провести следующую HR-сессию?,в москве,1.0
29660,•Где вы хотите провести следующую HR-сессию?,в сочи,1.0
29661,•Где вы хотите провести следующую HR-сессию?,сочи,1.0
29662,•Где вы хотите провести следующую HR-сессию?,окбм,1.0


In [474]:
index = pd.MultiIndex.from_arrays(df_all.iloc[:, :-1].values.T, names=('question', 'answer'))
df_grouped = pd.DataFrame({'count' : df_all.iloc[:, -1:].values.T[0]}, index=index)

In [475]:
df_grouped

count
question                                     answer                               
Что вы сможете использовать в своей работе?  формулировать проблему, 5 поче    1.0
                                             поиск потерь                      1.0
                                             правило 5 «почему» :)             1.0
                                             умение работать в команде         1.0
                                             картирование и анализ             1.0
...                                                                            ...
•Где вы хотите провести следующую HR-сессию? в москве                          1.0
                                             в сочи                            1.0
                                             сочи                              1.0
                                             окбм                              1.0
                                             сочи                              1.0

[29664 rows x 1 columns]

In [492]:
def get_abbreviations(f):
    abbreviations = dict()
    with open(f) as file:
        for line in file.readlines():
            key, value = map(str.strip, line.split('->'))
            abbreviations[key] = value
    return abbreviations

In [493]:
def replace_abbreviations(sentence):
    abbreviations = get_abbreviations('../data/слова.txt')
    words = sentence.lower().split()
    for i in range(len(words)):
        if words[i] in abbreviations:
            words[i] = abbreviations[words[i]]
    return ' '.join(words)

In [494]:
replace_abbreviations('я не оч, все раб. время в it отдыхаю, чил, зе бест')

'я не оч, все раб. время в информационные технологии отдыхаю, чил, зе отлично'

In [281]:
df_labeled = pd.read_csv('../data/df_labeled.csv').drop(['Unnamed: 0', 'id', 'cluster', 'count', 'corrected'], axis=1)

In [282]:
df_labeled

,question,answer,sentiment
0,Вопрос 7. Что должны сделать Лидеры безопаснос...,создание раб.групп для контрол,neutrals
1,Вопрос 7. Что должны сделать Лидеры безопаснос...,сократить дублирование,positives
2,Вопрос 7. Что должны сделать Лидеры безопаснос...,травмы и работать над ней,negatives
3,Вопрос 7. Что должны сделать Лидеры безопаснос...,убедить работника что его жиз,neutrals
4,Вопрос 7. Что должны сделать Лидеры безопаснос...,улучшать качество работы,positives
...,...,...,...
2804,Из какого вы города?,трёхгорный,neutrals
2805,Из какого вы города?,троицк,neutrals
2806,Из какого вы города?,шумиха,neutrals
2807,Из какого вы города?,электросталь,neutrals


In [283]:
from dostoevsky.tokenization import RegexTokenizer
from dostoevsky.models import FastTextSocialNetworkModel

In [284]:
tokenizer = RegexTokenizer()
model = FastTextSocialNetworkModel(tokenizer=tokenizer)

In [285]:
def get_keys(list_of_dicts):
    keys = []
    for d in list_of_dicts:
        for key in d:
            keys.append(key)
    return keys

In [286]:
messages = df_all['answer'].values
list_of_sentiments = model.predict(messages, k=1)
df_all['sentiment'] = [sentiment if sentiment != 'speech' else 'positive' for sentiment in get_keys(list_of_sentiments)]

In [444]:
model.predict(['сила выносливость'], k=3)

[{'skip': 0.5621865391731262,
  'neutral': 0.2689514458179474,
  'negative': 0.051855459809303284}]

In [291]:
df_all[df_all['sentiment'] == 'skip']

,question,answer,count,sentiment
132,"Что для вас ""культура доверия"" в организации?",отсутствие наказаний за ошибки,1.0,skip
198,Что хорошего сделал за день?,макет,1.0,skip
360,Ваша главная эмоции сейчас?,страшно чутка,1.0,skip
449,"Считаете ли вы, что ТТТ могут помочь?","да, но…",1.0,skip
451,"Считаете ли вы, что ТТТ могут помочь?","да, но",1.0,skip
453,"Считаете ли вы, что ТТТ могут помочь?",хотелось бы,1.0,skip
479,"Считаете ли вы, что ТТТ могут помочь?",хотелось бы,1.0,skip
714,Что сейчас в Вашей организации является самым ...,плакаты,1.0,skip
826,Что планируете применять из пройденного обучен...,диагностику,1.0,skip
829,Что планируете применять из пройденного обучен...,"диагностику, гибкость",1.0,skip


In [469]:
df_all['question'].values[29233]

'С какими трудностями Вы сталкиваетесь как кайдзен-лидер? Укажите Топ 3 проблемы.'

In [440]:
from spellchecker import SpellChecker

spell = SpellChecker(language='ru')

In [468]:
print(spell.candidates('оформл'))

{'формы', 'форма', 'форму', 'форме'}
